In [1]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")

# Initialize the tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples["article"], padding="max_length", truncation=True, max_length=512,return_tensors="pt")
    targets = tokenizer(examples["highlights"], padding="max_length", truncation=True, max_length=150, return_tensors="pt")
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": targets["input_ids"]}

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
dataloader = DataLoader(tokenized_dataset, batch_size=8, shuffle=True)

/anaconda/envs/LLM3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset cnn_dailymail (/home/azureuser/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-db1000349a810cfb.arrow


In [2]:
#add wandb logging
# %pip install wandb
import wandb
wandb.init(project="T5-distill-project")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shubsoni. Use `wandb login --relogin` to force relogin


In [3]:
# Load the teacher model
teacher_model = T5ForConditionalGeneration.from_pretrained("t5-small")
# Create the student model
student_config = T5Config.from_pretrained("t5-small", d_model=128, d_ff=512, d_kv=64, num_layers=2)
student_model = T5ForConditionalGeneration(student_config)

In [4]:
tokenized_dataset.set_format(type="torch")
tokenized_dataset[0]['input_ids'].shape

torch.Size([512])

In [5]:
from torch.optim import Adam
from tqdm import tqdm  # Progress bar
# Set up the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def convert_to_tensor(data):
    """Helper function to convert data to tensor if it's not already a tensor."""
    return torch.stack(data).to(device) if isinstance(data[0], torch.Tensor) else torch.tensor(data).to(device)

step =0
teacher_model.to(device)
student_model.to(device)
optimizer = Adam(student_model.parameters())  # Set hyperparameters as needed
# Train the student model
for epoch in range(3):
    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        # Convert lists of tensors to a single tensor and send to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids, 
                                           attention_mask=attention_mask,
                                           decoder_input_ids=input_ids,
                                           decoder_attention_mask=attention_mask).logits
        
        student_logits = student_model(input_ids=input_ids, 
                                       attention_mask=attention_mask, 
                                       decoder_input_ids=input_ids,
                                       decoder_attention_mask=attention_mask).logits

        # Calculate the knowledge distillation loss using the logits from the teacher and student models
        loss = torch.nn.functional.kl_div(torch.nn.functional.log_softmax(student_logits, dim=-1),
                                          torch.nn.functional.softmax(teacher_logits, dim=-1),
                                          reduction='batchmean')

        # Backpropagate the loss and update the model's weights
        loss.backward()
        optimizer.step()
    

        if step %100 == 0:
            print(f"Loss {loss.item()}")
            wandb.log({"loss": loss.item(), "step": step})  # Log loss to wandb
        step+=1

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/359 [00:00<?, ?it/s]

  1%|█████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 2/359 [00:00<02:15,  2.63it/s]

Loss 5781.11181640625


 28%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 102/359 [00:17<00:42,  5.98it/s]

Loss 1378.909912109375


...

 51%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 184/359 [00:30<00:29,  5.97it/s]

Loss 853.1483764648438


 79%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                                                                        | 284/359 [00:47<00:12,  5.98it/s]

Loss 847.409423828125


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 359/359 [01:00<00:00,  5.98it/s]


In [6]:
student_model_path = "./T5-distilled-student"
tokenizer.save_pretrained(student_model_path)
student_model.save_pretrained(student_model_path)